In [0]:
from mlflow.models.signature import infer_signature
from sklearn.metrics import roc_auc_score
import mlflow.sklearn
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from mlflow.tracking.client import MlflowClient
import time

# Data prep
white_wine = pd.read_csv("/dbfs/databricks-datasets/wine-quality/winequality-white.csv", sep=";")
red_wine = pd.read_csv("/dbfs/databricks-datasets/wine-quality/winequality-red.csv", sep=";")

red_wine['is_red'] = 1
white_wine['is_red'] = 0

data = pd.concat([red_wine, white_wine], axis=0)

# Remove spaces from column names
data.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

high_quality = (data.quality >= 7).astype(int)
data.quality = high_quality

data.dropna(inplace=True)
data.reset_index(drop=True,inplace=True)
    
train, test = train_test_split(data, random_state=123)
X_train = train.drop(["quality"], axis=1)
X_test = test.drop(["quality"], axis=1)
y_train = train.quality
y_test = test.quality

# Train and log model
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

2024/04/11 12:01:59 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/04/11 12:01:59 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.
2024/04/11 12:01:59 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.ml.
/databricks/python/lib/python3.9/site-packages/mlflow/models/signature.py:130: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-m

Out[1]: RandomForestClassifier(random_state=42)